In [12]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, TensorDataset
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, ConfusionMatrixDisplay
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from torchsummary import summary
from sklearn.model_selection import train_test_split

데이터 로드 및 확인

In [2]:
data = pd.read_csv('https://raw.githubusercontent.com/mwaskom/seaborn-data/master/penguins.csv')  # CSV 파일 경로
data

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,MALE
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,FEMALE
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,FEMALE
3,Adelie,Torgersen,NaN,NaN,NaN,NaN,NaN
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,FEMALE
...,...,...,...,...,...,...,...
339,Gentoo,Biscoe,NaN,NaN,NaN,NaN,NaN
340,Gentoo,Biscoe,46.8,14.3,215.0,4850.0,FEMALE
341,Gentoo,Biscoe,50.4,15.7,222.0,5750.0,MALE
342,Gentoo,Biscoe,45.2,14.8,212.0,5200.0,FEMALE


데이터 엔코딩

In [3]:
# 원-핫 인코딩을 적용할 범주형 변수 리스트
categorical_columns = [
    'species', 'island', 'bill_length_mm', 'bill_depth_mm', 
    'flipper_length_mm', 'body_mass_g', 'sex'
]

# NaN 값은 우선 0으로 채움 (또는 필요시 평균으로 채워도 됨)
data[categorical_columns] = data[categorical_columns].fillna(0)

# 원-핫 인코딩 적용 (drop_first=True로 다중공선성 방지)
data_encoded = pd.get_dummies(data, columns=categorical_columns, drop_first=True)

# 남은 NaN 처리 (연속형 변수들 포함)
data_encoded = data_encoded.fillna(0)

# 모든 컬럼을 float32로 변환 (PyTorch 호환용)
data_encoded = data_encoded.astype('float32')

# 결과 확인
print(data_encoded)

     species_Chinstrap  species_Gentoo  island_Dream  island_Torgersen  \
0                  0.0             0.0           0.0               1.0   
1                  0.0             0.0           0.0               1.0   
2                  0.0             0.0           0.0               1.0   
3                  0.0             0.0           0.0               1.0   
4                  0.0             0.0           0.0               1.0   
..                 ...             ...           ...               ...   
339                0.0             1.0           0.0               0.0   
340                0.0             1.0           0.0               0.0   
341                0.0             1.0           0.0               0.0   
342                0.0             1.0           0.0               0.0   
343                0.0             1.0           0.0               0.0   

     bill_length_mm_32.1  bill_length_mm_33.1  bill_length_mm_33.5  \
0                    0.0                 

타겟 변수

In [8]:
# 데이터와 타겟 분리
X = data.drop('species', axis=1).values
y = data['species'].values

데이터 분할

In [9]:
# 데이터 분할: 훈련 데이터와 테스트 데이터 80:20 비율로 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

데이터 스케일링

In [14]:
# Standardize the data
scaler = StandardScaler()
X = scaler.fit_transform(X)

ValueError: could not convert string to float: 'Torgersen'